In [10]:
import numpy as np
import scipy.linalg as LA
from scipy.optimize import lsq_linear

In [11]:
def calibrate_uv_xyz_transform(calibration_file: str, calibration_type: str = "correlated") -> np.ndarray:
    if calibration_type == "correlated":
        U, V, X, Y, Z = np.loadtxt(calibration_file,skiprows=2, unpack=True, delimiter=",")
        image_space = np.array([U,V]).T
        world_space = np.array([X, Y, Z]).T
        R = create_regressor(image_space)
        sol = lsq_linear(LA.block_diag(R,R,R) ,np.append(X, [Y,Z]))
        Θ = sol.x.reshape(10,3, order="F")
        return Θ
    elif calibration_type == "params":
        Θ = np.loadtxt(calibration_file,skiprows=1, delimiter=",")
        return Θ

def create_regressor(uv: np.ndarray) -> np.ndarray:
    Φ = np.transpose([np.ones_like(uv[:,0]),
                        uv[:,0], uv[:,0]**2, uv[:,0]**3, 
                        uv[:,1], uv[:,1]**2, uv[:,1]**3, 
                        uv[:,0]*uv[:,1], 
                        uv[:,0]**2 *uv[:,1], 
                        uv[:,0]*uv[:,1]**2 ])
    return Φ 

def project_image(uv: np.ndarray, Θ: np.ndarray) -> np.ndarray:
    Φ = create_regressor(uv)
    P = Φ @ Θ
    return P

In [24]:
Θ = calibrate_uv_xyz_transform("calibration_data.csv")

In [13]:
U, V, X, Y, Z = np.loadtxt("calibration_validation.csv",skiprows=2, unpack=True, delimiter=",")
image_space_validation = np.array([U,V]).T
world_space_validation = np.array([X, Y, Z]).T

In [14]:
projected_validation = project_image(image_space_validation,Θ)

np.average(world_space_validation - projected_validation, axis=0)

array([-2.41824324e-02, -1.67781328e-02, -1.43303345e-17])

In [25]:
test_point = np.array([[896,564]])
print(create_regressor(test_point)@Θ)

[[ 5.19586538e+00  1.42489943e+00 -6.38274677e-17]]
